In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("./results.csv")
df

## Let us separate SAT from UNSAT instances

In [ ]:
paladinus_incorrect = ["tireworld-truck", "earth_observation"]
df_unsat = df.query(f"solved==True and SAT=='False' and planner=='paladinus-4h' and domain not in @paladinus_incorrect")
df_unsat[["domain", "instance"]]

In [ ]:
df_sat = df[~(df.domain.isin(df_unsat.domain) & df.instance.isin(df_unsat.instance))]
df_sat.to_csv("sat_results.csv")
df_sat

df_unsat_1 = df[(df.domain.isin(df_unsat.domain) & df.instance.isin(df_unsat.instance))]
df_unsat_1.to_csv("unsat_results.csv")

## Compute coverage

In [ ]:
df_solved = df_sat.query(f"timeout==False and memoryout==False")

df_sat_counts = df_sat.query("planner=='paladinus-4h'").groupby(["domain"]).count()[["instance"]]
df_sat_counts.rename(columns={"instance": "total_count"}, inplace=True)
df_sat_counts


df_solved_count = df_solved.groupby(["domain", "planner"]).count()[["instance"]].reset_index()
df_solved_count.rename(columns={"instance": "count"}, inplace=True)
df_coverage = df_solved_count.merge(df_sat_counts, left_on="domain", right_on="domain", how="inner")
df_coverage["coverage"] = df_coverage["count"]/df_coverage["total_count"]
df_coverage.to_csv("coverage.csv")
df_coverage

## Instance ranking!

In [ ]:
df_solved.drop(columns=["timeout", "solved", "SAT", "memoryout"], inplace=True)
df_solved.time = np.round(df_solved.time, 3)
df_solved.head()

In [ ]:
df_min_time = df_solved.groupby(["domain", "instance"]).min("time").reset_index()
df_min_time.time = np.round(df_min_time.time, 3)
df_min_time.rename(columns={"memory": "min_time_memory", "time": "min_time", "policysize": "min_time_policy"}, inplace=True)

df_max_time  = df_solved.groupby(["domain", "instance"]).max("time").reset_index()
df_max_time.time = np.round(df_max_time.time, 3)
df_max_time.rename(columns={"memory": "max_time_memory", "time": "max_time", "policysize": "max_time_policy"}, inplace=True)


df_solved_min = df_solved.merge(df_min_time, left_on=["domain", "instance"], right_on=["domain", "instance"], how="inner")
df_solved_min_max = df_solved_min.merge(df_max_time, left_on=["domain", "instance"], right_on=["domain", "instance"], how="inner")
df_solved_min_max

In [ ]:
df_solved_min_max["time_diff_min"] = df_solved_min_max["time"] - df_solved_min_max["min_time"]
df_solved_min_max["time_diff_max"] = df_solved_min_max["time"] - df_solved_min_max["max_time"]
df_solved_min_max["fastest"] = np.where(df_solved_min_max["time_diff_min"] == 0, 1, 0)
df_solved_min_max["slowest"] = np.where(df_solved_min_max["time_diff_max"] == 0, 1, 0)
df_solved_min_max

In [ ]:
df_solved_min_max.query("domain=='spiky-tireworld' and instance=='p04'")

In [ ]:
df_fastest = df_solved_min_max.groupby(["domain", "planner"]).sum("fastest")
df_fastest.drop(columns=["min_time_memory", "min_time_policy", "max_time", "max_time_memory", "max_time_policy", "time_diff_max", "slowest"], inplace=True)
df_fastest.reset_index(inplace=True)
df_fastest.query("domain=='doors'")

In [ ]:
df_fastest = df_solved_min_max.groupby(["domain", "planner"]).sum("fastest")
df_fastest.drop(columns=["min_time_memory", "min_time_policy", "max_time", "max_time_memory", "max_time_policy", "time_diff_max", "slowest"], inplace=True)
df_fastest.reset_index(inplace=True)
df_fastest.query("domain=='islands'")

In [ ]:
df_fastest = df_solved_min_max.groupby(["domain", "planner"]).sum("fastest")
df_fastest.drop(columns=["min_time_memory", "min_time_policy", "max_time", "max_time_memory", "max_time_policy", "time_diff_max", "slowest"], inplace=True)
df_fastest.reset_index(inplace=True)
df_fastest.query("domain=='miner'")

In [ ]:
df_fastest = df_solved_min_max.groupby(["domain", "planner"]).sum("fastest")
df_fastest.drop(columns=["min_time_memory", "min_time_policy", "max_time", "max_time_memory", "max_time_policy", "time_diff_max", "slowest"], inplace=True)
df_fastest.reset_index(inplace=True)
df_fastest.query("domain=='spiky-tireworld'")

In [ ]:
df_fastest = df_solved_min_max.groupby(["domain", "planner"]).sum("fastest")
df_fastest.drop(columns=["min_time_memory", "min_time_policy", "max_time", "max_time_memory", "max_time_policy", "time_diff_max", "slowest"], inplace=True)
df_fastest.reset_index(inplace=True)
df_fastest.query("domain=='tireworld-truck'")

In [ ]:
df_mean_time = df_solved.groupby(["domain", "planner"]).mean("time").reset_index()
df_mean_time

## New Domains

In [ ]:
new_domains = ["miner", "islands", "spiky-tireworld", "doors", "tireworld-truck"]
df_new_domains = df.query(f"domain in @new_domains")
df_new_domains

In [ ]:
df_new_domain_counts = df_new_domains.query("planner=='paladinus-4h'").groupby(["domain"]).count()[["instance"]]
df_new_domain_counts.rename(columns={"instance": "total_count"}, inplace=True)
df_new_domain_counts

In [ ]:
df_timed_out = df_new_domains.query(f"timeout==True or memoryout==True")
df_solved = df_new_domains.query(f"timeout==False and memoryout==False")
df_solved.to_csv("new_domains.csv")

In [ ]:
df_solved_count = df_solved.groupby(["domain", "planner"]).count()[["instance"]].reset_index()
df_solved_count.rename(columns={"instance": "count"}, inplace=True)
df_solved_all = df_solved_count.merge(df_new_domain_counts, left_on="domain", right_on="domain", how="inner")
df_solved_all["coverage"] = df_solved_all["count"]/df_solved_all["total_count"]
df_solved_all.to_csv("new_domains_coverage.csv")

In [ ]:
model_planners = ["asp1-4h", "asp2-backbone-4h", "minisat-4h", "glucose-4h"]
df_model_planners = df_solved.query(f"planner in @model_planners")
df_model_planners

In [ ]:
df_policy = df_model_planners.pivot(index=["domain", "instance"], columns=["planner"], values="policysize")
df_policy.reset_index(inplace=True)
df_policy.rename(columns={"asp1-4h":"A1", "asp2-backbone-4h": "A2", "glucose-4h":"FG", "minisat-4h": "FM"}, inplace=True)
df_policy

In [ ]:
df_policy.query(f"A1 != A2")

In [ ]:
df_policy.query(f"A1 != FG")

In [ ]:
df_solved